<a href="https://colab.research.google.com/github/currencyfxjle/Pandas-y-NumPy-en-Pyhton/blob/main/repository_pytorch_tensor_neural_network_epoch_regloss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import scipy.stats as stats

# Download the data for USDMXN=X within the specified period
usdmxn = yf.Ticker("USDMXN=X").history(start="2019-10-01", end="2020-06-01")

# Normalize the data
scaler = MinMaxScaler()
usdmxn[["Open", "High", "Low", "Close", "Volume"]] = scaler.fit_transform(usdmxn[["Open", "High", "Low", "Close", "Volume"]])

# Split the data into training, validation, and testing sets
q_80 = int(len(usdmxn) * .8)
q_90 = int(len(usdmxn) * .9)

train = usdmxn.iloc[:q_80]
val = usdmxn.iloc[q_80:q_90]
test = usdmxn.iloc[q_90:]

# Convert data to PyTorch tensors
predictors = ["Open", "High", "Low", "Volume"]

X_train = torch.FloatTensor(train[predictors].values)
y_train = torch.FloatTensor(train["Close"].values)

X_val = torch.FloatTensor(val[predictors].values)
y_val = torch.FloatTensor(val["Close"].values)

X_test = torch.FloatTensor(test[predictors].values)
y_test = torch.FloatTensor(test["Close"].values)

model = StockPredictor(input_dim=4, hidden_dim=50)

# Training parameters
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 200

# Train the deep learning model
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred_train = model(X_train).squeeze()
    loss = criterion(y_pred_train, y_train)
    loss.backward()
    optimizer.step()

# Predict for training, validation, and test sets
with torch.no_grad():
    y_pred_train = model(X_train).squeeze()
    y_pred_val = model(X_val).squeeze()
    y_pred_test = model(X_test).squeeze()

# De-normalize predictions for plotting
y_pred_train = scaler.inverse_transform(np.concatenate([X_train, y_pred_train.unsqueeze(1).numpy()], axis=1))[:, -1]
y_pred_val = scaler.inverse_transform(np.concatenate([X_val, y_pred_val.unsqueeze(1).numpy()], axis=1))[:, -1]
y_pred_test = scaler.inverse_transform(np.concatenate([X_test, y_pred_test.unsqueeze(1).numpy()], axis=1))[:, -1]

# De-normalize actuals
y_train = scaler.inverse_transform(np.concatenate([X_train, y_train.unsqueeze(1).numpy()], axis=1))[:, -1]
y_val = scaler.inverse_transform(np.concatenate([X_val, y_val.unsqueeze(1).numpy()], axis=1))[:, -1]
y_test = scaler.inverse_transform(np.concatenate([X_test, y_test.unsqueeze(1).numpy()], axis=1))[:, -1]

# Plot the actual and predicted prices
plt.figure(figsize=(15, 8))

# Train
plt.plot(train.index, y_train, color='blue', label="Train Actual")
plt.plot(train.index, y_pred_train, color='cyan', linestyle='dashed', label="Train Predicted")

# Validation
plt.plot(val.index, y_val, color='green', label="Validation Actual")
plt.plot(val.index, y_pred_val, color='limegreen', linestyle='dashed', label="Validation Predicted")

# Test
plt.plot(test.index, y_test, color='red', label="Test Actual")
plt.plot(test.index, y_pred_test, color='darkorange', linestyle='dashed', label="Test Predicted")

plt.legend()
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("Neural Network Predictions")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

def plot_residuals(actual, predicted, title, poly_degree=None):
    residuals = actual - predicted
    plt.figure(figsize=(12, 6))

    # Scatter plot of residuals
    plt.scatter(actual, residuals, color='blue', alpha=0.5)

    # Calculate mean and standard deviation of residuals
    mean_residuals = np.mean(residuals)
    std_residuals = np.std(residuals)

    # Add 1 standard deviation quantiles to the plot
    plt.axhline(y=mean_residuals - std_residuals, color='red', linestyle='--', label='-1 Std Dev')
    plt.axhline(y=mean_residuals, color='green', linestyle='--', label='Mean')
    plt.axhline(y=mean_residuals + std_residuals, color='orange', linestyle='--', label='+1 Std Dev')

    # Add 2 standard deviation quantiles to the plot
    plt.axhline(y=mean_residuals - 2 * std_residuals, color='purple', linestyle='--', label='-2 Std Dev')
    plt.axhline(y=mean_residuals + 2 * std_residuals, color='brown', linestyle='--', label='+2 Std Dev')

    # Add a polynomial trendline if poly_degree is provided
    if poly_degree is not None:
        coefficients = np.polyfit(actual, residuals, poly_degree)
        polynomial = np.poly1d(coefficients)
        trendline_residuals = polynomial(actual)
        plt.plot(actual, trendline_residuals, color='black', label=f'Poly Degree {poly_degree} Trendline')

    plt.xlabel('Actual Values')
    plt.ylabel('Residuals')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


# Residual Analysis
plot_residuals(y_train, y_pred_train, 'Residuals (Training)', poly_degree=4)
plot_residuals(y_train, y_pred_train, 'Neural Network Residuals (Training)')
plot_residuals(y_val, y_pred_val, 'Neural Network Residuals (Validation)')
plot_residuals(y_test, y_pred_test, 'Neural Network Residuals (Testing)')